# Preamble

In [1]:
import $file.sparksession
import sparksession._
import spark.implicits._
import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._, func._

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/03/04 15:09:57 INFO SparkContext: Running Spark version 2.4.3
20/03/04 15:09:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
20/03/04 15:09:58 INFO SparkContext: Submitted application: c759fc7c-e743-423a-bfd0-79c8689b52ee
20/03/04 15:09:58 INFO SecurityManager: Changing view acls to: jserrano
20/03/04 15:09:58 INFO SecurityManager: Changing modify acls to: jserrano
20/03/04 15:09:58 INFO SecurityManager: Changing view acls groups to: 
20/03/04 15:09:58 INFO SecurityManager: Changing modify acls groups to: 
20/03/04 15:09:58 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jserrano); groups with view permissions: Set(); users  with modify permissions: Set(jserrano); groups with modify permissions: Set()
20/03/04 15:09:59 INFO Utils: Successfully started service

20/03/04 15:10:00 INFO SparkContext: Added JAR file:/Users/jserrano/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/io/get-coursier/interface/0.0.12/interface-0.0.12.jar at spark://10.0.84.117:51020/jars/interface-0.0.12.jar with timestamp 1583331000438
20/03/04 15:10:00 INFO SparkContext: Added JAR file:/Users/jserrano/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0-sources.jar at spark://10.0.84.117:51020/jars/scala-collection-compat_2.12-2.0.0-sources.jar with timestamp 1583331000440
20/03/04 15:10:00 INFO SparkContext: Added JAR file:/Users/jserrano/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0.jar at spark://10.0.84.117:51020/jars/scala-collection-compat_2.12-2.0.0.jar with timestamp 1583331000442
20/03/04 15:10:00 INFO SparkContext: Added JAR file:/Users/jserrano/Library/

Spark UI

import $file.$           

import sparksession._

import spark.implicits._

import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._, func._

# On `DataFrame`s

https://spark.apache.org/docs/latest/sql-programming-guide.html

We can create datasets from external data sources using different formats, e.g. Json, parquet, CSV, etc. 

In [7]:
val people: DataFrame = spark.read.json("data/people.json")

json at cmd6.sc:1

1 / 1

people: DataFrame = [age: bigint, name: string]

Note that we created a `DataFrame`, not a `Dataset`. Dataframes are like datasets, i.e. programs to generate distributed data sets, but *dynamically typed*. This means that, at compile time, Scala only knows that a dataframe consists of `Row`s.

In [10]:
val ds: Dataset[Int] = List(1,2,3).toDS
ds.collect

ds: Dataset[Int] = [value: int]
res9_1: Array[Int] = Array(1, 2, 3)

In [11]:
people.collect

collect at cmd10.sc:1

1 / 1

res10: Array[Row] = Array(
  [2000,YiHui],
  [2000,Javier],
  [2003,Gabriel],
  [2004,Noelia]
)

In fact, a `DataFrame` is defined as an alias of `Dataset`: 

In [12]:
val peopleDs: Dataset[Row] = people

peopleDs: Dataset[Row] = [age: bigint, name: string]

But the type of the information to be processed is there! 

In [14]:
people.schema
people.printSchema

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



res13_0: StructType = StructType(
  StructField("age", LongType, true, {}),
  StructField("name", StringType, true, {})
)

and we can convert a dataframe into a dataset: 

In [28]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Person(name: String, age: Long)

val peopleDs: Dataset[Person] = people.as[Person]

defined class Person
peopleDs: Dataset[Person] = [age: bigint, name: string]

In [29]:
lazy val i: Int = {println("hola"); 1}

i: Int = [lazy]

In [23]:
i + 1


hola


res22: Int = 2

In [26]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Person(name1: String, age: Long)

// def peopleDs: Dataset[Person1] = people.as[Person1]
people.as[Person]

: 

In [27]:
peopleDs

: 

In [18]:
val i: Int = 0
val j: String = i.toString

i
j


i: Int = 0
j: String = "0"
res17_2: Int = 0
res17_3: String = "0"

In [15]:
peopleDs.show
people.show

show at cmd14.sc:1

1 / 1

+----+-------+
| age|   name|
+----+-------+
|2000|  YiHui|
|2000| Javier|
|2003|Gabriel|
|2004| Noelia|
+----+-------+



show at cmd14.sc:2

1 / 1

+----+-------+
| age|   name|
+----+-------+
|2000|  YiHui|
|2000| Javier|
|2003|Gabriel|
|2004| Noelia|
+----+-------+



# Untyped transformations

The `Dataset` API includes a section on _untyped transformations_. These are transformations that are not defined over the Scala types but over the inner Spark SQL types (i.e. `StructType`s). More exactly, these could be named *dynamically typed transformations*.

These transformations are in close corresponde with their SQL counterparts: `SELECT`, `WHERE`, `GROUP BY`, `FROM`, etc. 

### The `select` transformation

For instance, the equivalent to the `map` typed transformation is `select`: 

In [30]:
val ds: Dataset[String] = peopleDs.map(_.name)
ds.collect
ds.show
ds.explain

collect at cmd29.sc:2

1 / 1

show at cmd29.sc:3

1 / 1

+-------+
|  value|
+-------+
|  YiHui|
| Javier|
|Gabriel|
| Noelia|
+-------+

== Physical Plan ==
*(1) SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false) AS value#103]
+- *(1) MapElements ammonite.$sess.cmd29$Helper$$Lambda$5286/96895532@56768c7b, obj#102: java.lang.String
   +- *(1) DeserializeToObject newInstance(class ammonite.$sess.cmd27$Helper$Person), obj#101: ammonite.$sess.cmd27$Helper$Person
      +- *(1) FileScan json [age#48L,name#49] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/Users/jserrano/Documents/spark-intro/data/people.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:bigint,name:string>


ds: Dataset[String] = [value: string]
res29_1: Array[String] = Array("YiHui", "Javier", "Gabriel", "Noelia")

In [31]:
val df: DataFrame = 
    people.select($"name")
df.collect
df.show
df.schema

collect at cmd30.sc:3

1 / 1

show at cmd30.sc:4

1 / 1

+-------+
|   name|
+-------+
|  YiHui|
| Javier|
|Gabriel|
| Noelia|
+-------+



df: DataFrame = [name: string]
res30_1: Array[Row] = Array([YiHui], [Javier], [Gabriel], [Noelia])
res30_3: StructType = StructType(StructField("name", StringType, true, {}))

Note that we lost the column label (`name`) in the case of the dataset transformation. This is not happening with `select`. Moreover, we have more control over the resulting schema: 

In [37]:
val t3: Tuple3[String, Int, Boolean] = ("",1,false)


t3: (String, Int, Boolean) = ("", 1, false)

In [40]:
t3._2
t3._1
t3._3

res39_0: Int = 1
res39_1: String = ""
res39_2: Boolean = false

In [35]:
peopleDs.map((p: Person) => (p.name, ((p.age: Int) + 1): Int, p.name.substring(0,3)))
    .show

show at cmd34.sc:1

1 / 1

+-------+----+---+
|     _1|  _2| _3|
+-------+----+---+
|  YiHui|2001|YiH|
| Javier|2001|Jav|
|Gabriel|2004|Gab|
| Noelia|2005|Noe|
+-------+----+---+



In [45]:
people.select(col("name"), 
              (col("age").+(lit(1))) as "age", 
              col("name").substr(0,3) as "prefix")
    .show

show at cmd44.sc:3

1 / 1

+-------+----+------+
|   name| age|prefix|
+-------+----+------+
|  YiHui|2001|   YiH|
| Javier|2001|   Jav|
|Gabriel|2004|   Gab|
| Noelia|2005|   Noe|
+-------+----+------+



In [48]:
people.select($"name", 
              ($"age" + 1) as "age", 
              $"name".substr(0,3) as "prefix")
    .show

show at cmd47.sc:3

1 / 1

+-------+----+------+
|   name| age|prefix|
+-------+----+------+
|  YiHui|2001|   YiH|
| Javier|2001|   Jav|
|Gabriel|2004|   Gab|
| Noelia|2005|   Noe|
+-------+----+------+



In [83]:
lazy val df = people.select($"name", 
              (concat($"name",lit("_a")): Column) as "age", 
              ($"name".substr(0,3): Column) as "prefix",
              $"age" > 2001 as "c2")

df: DataFrame = [name: string, age: string ... 2 more fields]

In [84]:
df.collect

collect at cmd83.sc:1

1 / 1

res83: Array[Row] = Array(
  [YiHui,YiHui_a,YiH,false],
  [Javier,Javier_a,Jav,false],
  [Gabriel,Gabriel_a,Gab,true],
  [Noelia,Noelia_a,Noe,true]
)

The [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$) contains dozens of column operators.

Note that _untyped_, or more properly, _dynamically typed_, character means that the Scala compiler won't complain if we choose a non-existent column:

In [78]:
val df: DataFrame = spark.read.json("data/people.json")
val df2: DataFrame = df.toDF("agee","nam")
df2.select($"nam")

json at cmd77.sc:1

1 / 1

df: DataFrame = [age: bigint, name: string]
df2: DataFrame = [agee: bigint, nam: string]
res77_2: DataFrame = [nam: string]

In [80]:
df.select($"nam").explain
df2.explain

: 

The error will be shown at runtime: 

In [71]:
df

json at cmd69.sc:1

1 / 1

res70: DataFrame = [nam: string]

On the contrary, the error in the dataset transformation manifests at compile-time:

In [76]:
(peopleDs: Dataset[Person]).map(_.nam)

cmd76.sc:1: value nam is not a member of cmd76.this.cmd27.Person
val res76 = (peopleDs: Dataset[Person]).map(_.nam)
                                              ^Compilation Failed

: 

### The `filter` transformation

This is the equivalent to the typed `filter` transformation:

In [82]:
peopleDs.filter(_.age > 2000)

res81: Dataset[Person] = [age: bigint, name: string]

In [81]:
people.filter($"age" > 2001)
    .show

show at cmd80.sc:1

1 / 1

+----+-------+
| age|   name|
+----+-------+
|2003|Gabriel|
|2004| Noelia|
+----+-------+



If we pass a column function not denoting a boolean value, we won't even get a run-time exception:

In [91]:
def df: DataFrame = 
    people.filter($"name" > "Javier")

defined function df

In [92]:
df.show

show at cmd91.sc:1

1 / 1

+----+------+
| age|  name|
+----+------+
|2000| YiHui|
|2004|Noelia|
+----+------+



### The `groupBy` transformation

In [93]:
val students: DataFrame = spark.read.json("data/students.json")

json at cmd92.sc:1

1 / 1

students: DataFrame = [degree: string, name: string]

In [94]:
students.groupBy($"degree").count.show

show at cmd93.sc:1

1 / 1

show at cmd93.sc:1

1 / 1

show at cmd93.sc:1

...

show at cmd93.sc:1

20 / 20

show at cmd93.sc:1

39 / 39

+------+-----+
|degree|count|
+------+-----+
|   MAT|    2|
|    II|    4|
|    IS|    1|
+------+-----+



In [137]:
students.groupBy($"degree").pivot($"name").count.show

pivot at cmd136.sc:1

1 / 1

pivot at cmd136.sc:1

64 / 64

pivot at cmd136.sc:1

...

show at cmd136.sc:1

20 / 20

show at cmd136.sc:1

39 / 39

+------+-------+------+------+-----+
|degree|Gabriel|Javier|Noelia|YiHui|
+------+-------+------+------+-----+
|   MAT|   null|     1|  null|    1|
|    II|      1|     1|     1|    1|
|    IS|   null|  null|     1| null|
+------+-------+------+------+-----+



In [94]:
students.groupBy($"degree").count.show

show at cmd93.sc:1

1 / 1

show at cmd93.sc:1

1 / 1

show at cmd93.sc:1

...

show at cmd93.sc:1

20 / 20

show at cmd93.sc:1

39 / 39

+------+-----+
|degree|count|
+------+-----+
|   MAT|    2|
|    II|    4|
|    IS|    1|
+------+-----+



### `Join` transformations

We already discussed joins, but we didn't mention that the resulting type of a join is a dataframe, not a dataset: 

In [95]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Student(name: String, degree: String)

defined class Student

In [98]:
peopleDs.join(students.as[Student], "name")

res97: DataFrame = [name: string, age: bigint ... 1 more field]

In [100]:
res97.show

run at ThreadPoolExecutor.java:1142

1 / 1

show at cmd99.sc:1

1 / 1

+-------+----+------+
|   name| age|degree|
+-------+----+------+
|  YiHui|2000|    II|
|  YiHui|2000|   MAT|
| Javier|2000|    II|
| Javier|2000|   MAT|
|Gabriel|2003|    II|
| Noelia|2004|    II|
| Noelia|2004|    IS|
+-------+----+------+



In [104]:
peopleDs.join(students.as[Student], 
              people.col("name").equalTo(students.col("name")))
    .drop(people.col("name"))
    .show

run at ThreadPoolExecutor.java:1142

1 / 1

show at cmd103.sc:3

1 / 1

+----+------+-------+
| age|degree|   name|
+----+------+-------+
|2000|    II|  YiHui|
|2000|   MAT|  YiHui|
|2000|    II| Javier|
|2000|   MAT| Javier|
|2003|    II|Gabriel|
|2004|    II| Noelia|
|2004|    IS| Noelia|
+----+------+-------+



In [113]:
people.join(students, 
              lit(false)) // people.col("name").equalTo(lit("")))
    .drop(people.col("name"))
    .show

+---+------+----+
|age|degree|name|
+---+------+----+
+---+------+----+



In [108]:
people.join(students,"name")
    .show

run at ThreadPoolExecutor.java:1142

1 / 1

show at cmd107.sc:1

1 / 1

+-------+----+------+
|   name| age|degree|
+-------+----+------+
|  YiHui|2000|    II|
|  YiHui|2000|   MAT|
| Javier|2000|    II|
| Javier|2000|   MAT|
|Gabriel|2003|    II|
| Noelia|2004|    II|
| Noelia|2004|    IS|
+-------+----+------+



# The problems of `Dataset`s

Datasets are nice because they are type safe, but, unfortunately, they are less efficient than data frames in several respects. This can be best shown by reading from parquet source files. 

Parquet is a _columnar_ format, which means that it stores physically data around columns, allowing us to read only data from a particular column without reading the entire row.

In [116]:
people.write.mode("overwrite").parquet("data/people.parquet")

parquet at cmd115.sc:1

1 / 1

In [118]:
spark.read.parquet("data/people.parquet").show

parquet at cmd117.sc:1

1 / 1

show at cmd117.sc:1

1 / 1

+----+-------+
| age|   name|
+----+-------+
|2000|  YiHui|
|2000| Javier|
|2003|Gabriel|
|2004| Noelia|
+----+-------+



### The `ReadSchema` optimization

Let's create a program that simply read the _name_ column of the people dataset:

In [123]:
List(Person("a",1), Person("b", 2)).toDS
    .map((t: Person) => t.name).explain

== Physical Plan ==
*(1) SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false) AS value#945]
+- *(1) MapElements ammonite.$sess.cmd122$Helper$$Lambda$6471/1725825756@5a64274c, obj#944: java.lang.String
   +- *(1) DeserializeToObject newInstance(class ammonite.$sess.cmd27$Helper$Person), obj#943: ammonite.$sess.cmd27$Helper$Person
      +- LocalTableScan [name#939, age#940L]


In [124]:
val ds: Dataset[String] = 
    spark.read.parquet("data/people.parquet").as[Person]
        .map(_.name)

parquet at cmd123.sc:2

1 / 1

ds: Dataset[String] = [value: string]

In [120]:
ds.explain

== Physical Plan ==
*(1) SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false) AS value#923]
+- *(1) MapElements ammonite.$sess.cmd118$Helper$$Lambda$6460/1380480315@26e34f85, obj#922: java.lang.String
   +- *(1) DeserializeToObject newInstance(class ammonite.$sess.cmd27$Helper$Person), obj#921: ammonite.$sess.cmd27$Helper$Person
      +- *(1) FileScan parquet [age#913L,name#914] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/Users/jserrano/Documents/spark-intro/data/people.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:bigint,name:string>


which works as intended: 

In [125]:
ds.show

show at cmd124.sc:1

1 / 1

+-------+
|  value|
+-------+
|  YiHui|
| Javier|
|Gabriel|
| Noelia|
+-------+



We have a problem, however: 

In [126]:
ds.explain

== Physical Plan ==
*(1) SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false) AS value#958]
+- *(1) MapElements ammonite.$sess.cmd123$Helper$$Lambda$6474/322703264@37c1d315, obj#957: java.lang.String
   +- *(1) DeserializeToObject newInstance(class ammonite.$sess.cmd27$Helper$Person), obj#956: ammonite.$sess.cmd27$Helper$Person
      +- *(1) FileScan parquet [age#948L,name#949] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/Users/jserrano/Documents/spark-intro/data/people.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:bigint,name:string>


As we can see, the plan includes the directive `ReadSchema: struct<age:bigint,name:string>`, which generates a query to scan the full schema of the parquet file. But we just want to read the names! We can create an optimun program using dataframes:

In [134]:
val df: DataFrame = 
    spark.read.parquet("data/people.parquet")
        .select($"name", concat($"name", lit("aaa")), $"age" + 1)

parquet at cmd133.sc:2

1 / 1

df: DataFrame = [name: string, concat(name, aaa): string ... 1 more field]

which works similarly: 

In [135]:
df.show

show at cmd134.sc:1

1 / 1

+-------+-----------------+---------+
|   name|concat(name, aaa)|(age + 1)|
+-------+-----------------+---------+
|  YiHui|         YiHuiaaa|     2001|
| Javier|        Javieraaa|     2001|
|Gabriel|       Gabrielaaa|     2004|
| Noelia|        Noeliaaaa|     2005|
+-------+-----------------+---------+



but more efficiently (note the the value of the `ReadSchema` directive):

In [136]:
df.explain

== Physical Plan ==
*(1) Project [name#995, concat(name#995, aaa) AS concat(name, aaa)#998, (age#994L + 1) AS (age + 1)#999L]
+- *(1) FileScan parquet [age#994L,name#995] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/Users/jserrano/Documents/spark-intro/data/people.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:bigint,name:string>


We can empirically check that it actually works using the Spark UI. First, we create a parquet file with enough rows and several columns:

In [ ]:
spark.range(0, 1000000)
    .select($"id" as "_1", lit(1) as "_2")
    .write.mode("overwrite").parquet("data/test")

Now, we read the second column using both datasets and dataframes, and check the Spark UI for the _Input Size_ field.

In [ ]:
val test = spark.read.parquet("data/test")
test.as[Tuple2[Long, Int]].map(_._2).collect

Using dataframes the input size is much lower since we only read the second column:

In [ ]:
test.select($"_2").collect

### The `PushedFilter` optimization

Let's consider the following equivalent dataset and dataframe programs: 

In [ ]:
val ds: Dataset[(Long, Int)] = 
    test.as[(Long, Int)]
        .filter(_._1 >= 999995)

val df: DataFrame = 
    test
        .filter($"_1" >= 999995)

Functionally, they are equivalent, but their performance differ significantly:

In [ ]:
df.collect
ds.collect

The explanation of this difference lies in another optimization applied by the Spark SQL compiler: the so-called push-down filter optimization. In the previous `ReadSchema` optimization, we skipped certain columns of the dataset; now, we skip rows and read only the ones we are interested in (those that satisfy the predicate). We can check if the push-down filter optimization is actually applied by inspecting the query plan. 

In [ ]:
df.explain
ds.explain

### The `PartitionFilters` optimization

Let's create a test file with an additional column: 

In [ ]:
spark.range(0, 1000000)
    .select($"id" as "_1", lit(1) as "_2", round(rand() * 10) mod lit(10) as "_3")
    .write.mode("overwrite").parquet("data/test")

In [ ]:
val test: DataFrame = spark.read.parquet("data/test")

In [ ]:
test.show

Let's suppose that we want to read data with value `_3` equal to `9.0`:

In [ ]:
test.filter($"_3" === lit(9.0)).show

A pushed filter optimization is created, but it would be better if we could just read directly those rows with the exact value for the thrid column. We can achieve that as follows:

In [ ]:
test.write.mode("overwrite").partitionBy("_3").parquet("data/testP")

As we can see, the parquet file is splitted into ten partitions. Now, if we just want to process data with a particular key, Spark will generate an optimun query: 

In [ ]:
val testP: DataFrame = spark.read.parquet("data/testP")

In [ ]:
testP.filter($"_3" === lit(9.0)).show

We can inspet the Spark UI to check that we read less data in the last action.